In [30]:
!pip install -qU llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-graph-stores-neo4j graspologic numpy==1.24.4 scipy==1.12.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [55]:
%pip install -qU llama-index-vector-stores-chroma
%pip install -qU llama-index-embeddings-huggingface
%pip install -qU llama-index-llms-ollama
%pip install -qU textwrap

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement textwrap (from versions: none)
ERROR: No matching distribution found for textwrap
Note: you may need to restart the kernel to use updated packages.


In [43]:
from llama_index.core import Settings, ChatPromptTemplate, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama

import chromadb

In [68]:
llm = Ollama(model="llama3.1")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

Settings.llm = llm
Settings.chunk_overlap = 5
Settings.chunk_size = 512
Settings.embed_model = embed_model

chroma_client = chromadb.PersistentClient(path="./data/chroma_db")
chroma_collection = chroma_client.get_or_create_collection("rag_chatbot")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader("./data").load_data()

index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
)
index.storage_context.persist(persist_dir="./data/vectorstore")

# query_engine = index.as_query_engine(similarity_top_k=1)
# response = query_engine.query("Tell me about Truc Nguyen?")
# display(Markdown(response.response))
# response.print_response_stream()

query = "List all programming skills of Truc Nguyen?"
chat_engine = index.as_chat_engine(similarity_top_k=10, chat_mode="context")
response = chat_engine.stream_chat(query)
for token in response.response_gen:
    print(token, end="")
# display(Markdown(response.response))

Based on the provided context, here are the programming skills listed for Truc Nguyen:

1. Programming languages:
	* C#
	* JavaScript (ES)/TypeScript
	* Dart
2. Backend technologies:
	* .NET Core
	* ASP.NET Core
	* ASP.NET WebAPI
3. Testing frameworks and libraries:
	* NUnit
	* MSTest with Moq
4. Frontend frameworks:
	* Angular
	* AngularJS
	* React
5. Other programming skills:
	* Unit testing
	* TDD (Test-Driven Development)
6. Databases:
	* SQL Server
	* SQLite

Let me know if you need any further assistance!

In [ ]:
message_templates = [
    ChatMessage(content="You are an expert system", role=MessageRole.SYSTEM),
    ChatMessage(
        content="Generate a short story about {topic}",
        role=MessageRole.USER,
    )
]

chat_template = ChatPromptTemplate(message_templates=message_templates)

messages = chat_template.format_messages(
    topic="Writing good CV to pass CV screening at FAANG companies"
)

stream_resp = llm.stream_chat(messages)
for r in stream_resp:
    print(r.delta, end="")